In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import pearsonr
from statsmodels.stats.anova import AnovaRM
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('t3fuds.csv')
df = df.replace(' ', '', regex = True)
df = df.replace('', np.NaN)
df = df.apply(pd.to_numeric, errors='coerce')

In [3]:
#create new df to input vars of interest from fullds
sub_use = pd.DataFrame()

#add participant ids
sub_use['id'] = df.unique_id

#add demographics from masterfile
demos = pd.read_csv('pt_demos.csv')
demos = demos.replace('-99', np.NaN)
ethnicities = {'1' : 'American Indian/Alaska Native', 
               '2' : 'Asian', 
               '3' : 'Black/African American', 
               '4' : 'Hispanic/Latino', 
               '5' : 'More than one race', 
               '6' : 'Native Hawaiian/Other Pacific Islander', 
               '7' : 'Unknown', 
               '8' : 'White'}
demos = demos.replace({"ethnicity" : ethnicities}) 
demos.sex = demos.sex.replace('1', 'M', regex=True)
demos.sex = demos.sex.replace('2', 'F', regex=True)

sub_use = pd.merge(sub_use, demos, how = 'left', left_on = 'id', right_on = 'unique_id')
sub_use.drop('unique_id', axis=1, inplace=True)

In [4]:
#pull and convert drug use variable
sub_use['t1_drug_use'] = df.y1s_drug.where(df.y1s_drug <= 3, 3)
sub_use['t2_drug_use'] = df.y2s_drug.where(df.y2s_drug <= 3, 3)
sub_use['t3_drug_use'] = df.y3s_drug.where(df.y3s_drug <= 3, 3)

In [5]:
#pull and convert caffine variables
def caff_dep1(row):
    if row['y1s_caff_3'] == 0:
        return 0
    elif row['y1s_caff_3'] == 1:
        return 2
    elif row['y1s_caff_3'] > 1:
        return 3
sub_use['t1_caff_dep'] = df.apply(lambda row: caff_dep1(row), axis=1)

def caff_dep2(row):
    if row['y2s_caff_3'] == 0:
        return 0
    elif row['y2s_caff_3'] == 1:
        return 2
    elif row['y2s_caff_3'] > 1:
        return 3
sub_use['t2_caff_dep'] = df.apply(lambda row: caff_dep2(row), axis=1)

def caff_dep3(row):
    if row['y3s_caff_3'] == 0:
        return 0
    elif row['y3s_caff_3'] == 1:
        return 2
    elif row['y3s_caff_3'] > 1:
        return 3
sub_use['t3_caff_dep'] = df.apply(lambda row: caff_dep3(row), axis=1)

In [6]:
#pull and convert nicotine variables
def nic_use1(row):
    if row['y1s_nic_2'] <= 2:
        return 0
    elif row['y1s_nic_2'] == 3:
        return 1
    elif row['y1s_nic_2'] > 3:
        return 3
sub_use['t1_nic_use'] = df.apply(lambda row: nic_use1(row), axis=1)

def nic_use2(row):
    if row['y2s_nic_2'] <= 2:
        return 0
    elif row['y2s_nic_2'] == 3:
        return 1
    elif row['y2s_nic_2'] > 3:
        return 3
sub_use['t2_nic_use'] = df.apply(lambda row: nic_use2(row), axis=1)

def nic_use3(row):
    if row['y3s_nic_2'] <= 2:
        return 0
    elif row['y3s_nic_2'] == 3:
        return 1
    elif row['y3s_nic_2'] > 3:
        return 3
sub_use['t3_nic_use'] = df.apply(lambda row: nic_use3(row), axis=1)

sub_use['t1_nic_dep'] = df.y1s_nic_3.where(df.y1s_nic_3 <= 3, 3)
sub_use['t2_nic_dep'] = df.y2s_nic_3.where(df.y2s_nic_3 <= 3, 3)
sub_use['t3_nic_dep'] = df.y3s_nic_3.where(df.y3s_nic_3 <= 3, 3)

In [7]:
#pull and convert alcohol vars
def alc_use1(row):
    if row['y1s_alc_use'] == 2:
        return 1
    elif row['y1s_alc_use'] == 3:
        return 2
    elif row['y1s_alc_use'] >= 6:
        return 3
    elif row['y1s_alc_use'] == 0 or 1:
        return 0
sub_use['t1_alc_use'] = df.apply(lambda row: alc_use1(row), axis=1)

def alc_use2(row):
    if row['y2s_alc_use'] == 2:
        return 1
    elif row['y2s_alc_use'] == 3:
        return 2
    elif row['y2s_alc_use'] >= 6:
        return 3
    elif row['y2s_alc_use'] == 0 or 1:
        return 0
sub_use['t2_alc_use'] = df.apply(lambda row: alc_use2(row), axis=1)

def alc_use3(row):
    if row['y3s_alc_use'] == 2:
        return 1
    elif row['y3s_alc_use'] == 3:
        return 2
    elif row['y3s_alc_use'] >= 6:
        return 3
    elif row['y3s_alc_use'] == 0 or 1:
        return 0
sub_use['t3_alc_use'] = df.apply(lambda row: alc_use3(row), axis=1)

sub_use['t1_alc_dep'] = df.y1s_alc_16
sub_use['t2_alc_dep'] = df.y2s_alc_16
sub_use['t3_alc_dep'] = df.y3s_alc_16

In [8]:
#lump use factor
sub_use['t1_use'] = sub_use.t1_drug_use + sub_use.t1_nic_use + sub_use.t1_alc_use
sub_use['t2_use'] = sub_use.t2_drug_use + sub_use.t2_nic_use + sub_use.t2_alc_use
sub_use['t3_use'] = sub_use.t3_drug_use + sub_use.t3_nic_use + sub_use.t3_alc_use

In [9]:
#pull and convert risk factors
sub_use['t1_pos_expect'] = df.y1s_pos_expec.apply(lambda x: x-1)
sub_use['t2_pos_expect'] = df.y2s_pos_expec.apply(lambda x: x-1)

sub_use['t1_neg_expect'] = df.y1s_neg_expec.apply(lambda x: x-1)
sub_use['t2_neg_expect'] = df.y2s_neg_expec.apply(lambda x: x-1)

sub_use['t1_cope'] = df.y1s_cope
sub_use['t2_cope'] = df.y2s_cope
sub_use['t3_cope'] = df.y3s_cope

sub_use['t1_conform'] = df.y1s_conform
sub_use['t2_conform'] = df.y2s_conform
sub_use['t3_conform'] = df.y3s_conform

In [10]:
#pull and convert stressful events
sub_use['t1_str'] = df.y1s_str_1
sub_use['t2_str'] = df.y2s_str_1
sub_use['t3_str'] = df.y3s_str_1

In [11]:
print('H1: Difficulty coping with issues is related to substance use')
#pearson's correlation
h1df = sub_use[['t1_cope','t2_cope','t3_cope','t1_use','t2_use','t3_use']]
h1df = h1df.dropna()
h1df['cope'] = h1df.t1_cope + h1df.t2_cope + h1df.t3_cope
h1df['use'] = h1df.t1_use + h1df.t2_use + h1df.t3_use
#checked assumptions
pearsonr(h1df.cope, h1df.use)
print('Results : r = 0.2641, p < 0.001; Null hypothesis rejected')

H1: Difficulty coping with issues is related to substance use
Results : r = 0.2641, p < 0.001; Null hypothesis rejected


In [12]:
print('H2: Positive expectancies of substance use at T1 are related to dependence at T3')
#pearson's correlation
h2df = sub_use[['t1_pos_expect','t3_caff_dep','t3_nic_dep','t3_alc_dep']]
h2df = h2df.dropna()
h2df['t3_dep'] = h2df.t3_caff_dep + h2df.t3_nic_dep + h2df.t3_alc_dep
#checked assumptions
pearsonr(h2df.t1_pos_expect, h2df.t3_dep)
print('Results : r = 0.2128, p < 0.001; Null hypothesis rejected')

H2: Positive expectancies of substance use at T1 are related to dependence at T3
Results : r = 0.2128, p < 0.001; Null hypothesis rejected


In [13]:
print('H3: Negative expectancies of substance use at T1 will impact amount of substance use at T2')
#two tailed two sample t test
h3df = sub_use[['t1_neg_expect', 't1_use', 't2_use']]
h3df = h3df.dropna()
h3df['use_change'] = h3df['t2_use'] - h3df['t1_use']

def expectancies(row):
    if row['t1_neg_expect'] <= 1:
        return 'low_neg_expect'
    elif row['t1_neg_expect'] >= 2:
        return 'high_neg_expect'
h3df['group'] = h3df.apply(lambda row: expectancies(row), axis=1)
low_neg_expect = h3df[h3df.group == 'low_neg_expect'].use_change
high_neg_expect = h3df[h3df.group == 'high_neg_expect'].use_change

#checked assumptions
stats.ttest_ind(high_neg_expect, low_neg_expect, equal_var=True)
print('Results : t = 2.57, p = 0.01; Null hypothesis rejected')

H3: Negative expectancies of substance use at T1 will impact amount of substance use at T2
Results : t = 2.57, p = 0.01; Null hypothesis rejected


In [14]:
print('H4: Experience of a stressful event will be associated with higher substance use')
#one tailed two sample t test
h4df = sub_use[['t1_str', 't2_str', 't3_str', 't3_use']]
h4df = h4df.dropna()
h4df['stress'] = h4df.t1_str + h4df.t2_str + h4df.t3_str
def stressor(row):
    if row['stress'] >= 1:
        return 'yes'
    else :
        return 'no'
h4df['group'] = h4df.apply(lambda row: stressor(row), axis=1)
stressor = h4df[h4df.group == 'yes'].t3_use
no_stressor = h4df[h4df.group == 'no'].t3_use

#checked assumptions
stats.ttest_ind(stressor, no_stressor, equal_var=True, alternative = 'greater')
print('Result : t = 7.11, p < 0.001; Null hypothesis rejected')

H4: Experience of a stressful event will be associated with higher substance use
Result : t = 7.11, p < 0.001; Null hypothesis rejected


In [15]:
print('H5: Substance use will change over time')
#Friedman Test
h5df = sub_use[['t1_use','t2_use', 't3_use']]
h5df = h5df.dropna()

#checked assumptions
stats.friedmanchisquare(h5df.t1_use, h5df.t2_use, h5df.t3_use)
print('Result : q = 94.10, p < 0.001; Null hypothesis rejected')

H5: Substance use will change over time
Result : q = 94.10, p < 0.001; Null hypothesis rejected


In [16]:
print('H6: Conformity will change over time')
#anova
h6df = sub_use[['t1_conform','t2_conform', 't3_conform']]
h6df = h6df.dropna()

#checked assumptions
fvalue, pvalue = stats.f_oneway(h6df.t1_conform, h6df.t2_conform, h6df.t3_conform)
print('Result : f = 8.19, p < 0.001; Null hypothesis rejected')

H6: Conformity will change over time
Result : f = 8.19, p < 0.001; Null hypothesis rejected
